In [1]:
import os, random, glob, pickle, collections
import numpy as np
import pandas as pd
import ujson as json
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('tf')

from skimage.data import imread
from skimage.io import imshow,imsave
import cv2
from skimage.util import crop
from skimage.transform import rotate
from skimage.transform import resize
import math

Using TensorFlow backend.


In [12]:
TRAIN_DIR = '../data/train/'
TEST_DIR = '../RFCN/JPEGImages/'
TRAIN_CROP_DIR = '../data/train_crop/'
TEST_CROP_DIR = '../data/test_stg1_crop/'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
CONF_THRESH = 0.8
ROWS = 224
COLS = 224
BatchSize = 128
LearningRate = 1e-4
le = LabelEncoder()
le.fit(FISH_CLASSES)
le.transform(FISH_CLASSES)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [3]:
#Loading data
import pickle

def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_CROP_DIR+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = Image.open(src)
    im = im.resize((COLS, ROWS), Image.BILINEAR)
    im = np.asarray(im)
    return im

if os.path.exists('../data/data_train_BBCrop_{}_{}.pickle'.format(ROWS, COLS)):
    print ('Exist data_train_BBCrop_{}_{}.pickle. Loading data from file.'.format(ROWS, COLS))
    with open('../data/data_train_BBCrop_{}_{}.pickle'.format(ROWS, COLS), 'rb') as f:
        data_train = pickle.load(f)
    X_train = data_train['X_train']
    y_train = data_train['y_train']
else:
    print ('Loading data from original images. Generating data_train_BBCrop_{}_{}.pickle.'.format(ROWS, COLS))
    
    files = []
    y_train = []

    for fish in FISH_CLASSES:
        fish_files = get_images(fish)
        files.extend(fish_files)

        y_fish = np.tile(fish, len(fish_files))
        y_train.extend(y_fish)
        #print("{0} photos of {1}".format(len(fish_files), fish))

    y_train = np.array(y_train)
    X_train = np.ndarray((len(files), ROWS, COLS, 3), dtype=np.uint8)

    for i, im in enumerate(files): 
        X_train[i] = read_image(TRAIN_CROP_DIR+im)
        if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

    #X_train = X_train / 255.
    #print(X_train.shape)

    # One Hot Encoding Labels
    y_train = le.transform(y_train)
    y_train = np_utils.to_categorical(y_train)
    
    #save data to file
    data_train = {'X_train': X_train,'y_train': y_train }

    with open('../data/data_train_BBCrop_{}_{}.pickle'.format(ROWS, COLS), 'wb') as f:
        pickle.dump(data_train, f)

#rescale
X_train = X_train / 255.
X_train = X_train.astype(np.float32)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=None, stratify=y_train)

Exist data_train_BBCrop_224_224.pickle. Loading data from file.


In [4]:
X_train.shape

(15583, 224, 224, 3)

In [5]:
#data preprocessing

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=[0.9,1.1],
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BatchSize, shuffle=True, seed=None)

valid_datagen = ImageDataGenerator(
    featurewise_center=True)
    #featurewise_std_normalization=True)
    #rescale=1./255
valid_datagen.fit(X_valid)   
valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size=BatchSize, shuffle=True, seed=None)

In [6]:
#callbacks

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        

model_checkpoint = ModelCheckpoint(filepath='./checkpoints/checkpoint2/weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        
learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)

tensorboard = TensorBoard(log_dir='./logs/log2', histogram_freq=0, write_graph=True, write_images=True)


In [ ]:
#Resnet50
#stg1 training

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
#x = Dropout(0.5)(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
#x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), init='glorot_normal', activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator, samples_per_epoch=len(X_train), nb_epoch=30, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=valid_generator, nb_val_samples=len(X_valid), nb_worker=3, pickle_safe=True)

Epoch 1/30
15488/15583 [============================>.] - ETA: 1s - loss: 1.6035 - acc: 0.4657

/opt/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from inf to 1.40994, saving model to ./checkpoints/checkpoint2/weights.000-1.4099.hdf5
15616/15583 [==============================] - 258s - loss: 1.5997 - acc: 0.4674 - val_loss: 1.4099 - val_acc: 0.7684
Epoch 2/30
15616/15583 [==============================] - 248s - loss: 1.2042 - acc: 0.6918 - val_loss: 1.3130 - val_acc: 0.7530
Epoch 3/30
15645/15583 [==============================] - 250s - loss: 1.0546 - acc: 0.7473 - val_loss: 1.2446 - val_acc: 0.7581
Epoch 4/30
15616/15583 [==============================] - 248s - loss: 0.9808 - acc: 0.7554 - val_loss: 1.1963 - val_acc: 0.7629
Epoch 5/30
15616/15583 [==============================] - 247s - loss: 0.9014 - acc: 0.7678 - val_loss: 1.1775 - val_acc: 0.7596
Epoch 6/30
 6528/15583 [===========>..................] - ETA: 115s - loss: 0.8754 - acc: 0.7782

In [10]:
#Resnet50
#stg2 training

# files = glob.glob('./checkpoints/*')
# val_losses = [float(f.split('-')[-1][:-5]) for f in files]
# index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file weights.028-0.4157.hdf5')
model = load_model('./checkpoints/checkpoint2/weights.028-0.4157.hdf5')

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
#164
for layer in model.layers[:142]:
   layer.trainable = False
for layer in model.layers[142:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=LearningRate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator, samples_per_epoch=len(X_train), nb_epoch=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=valid_generator, nb_val_samples=len(X_valid), nb_worker=3, pickle_safe=True)

Loading model from checkpoints file weights.028-0.4157.hdf5
0 input_1
1 zeropadding2d_1
2 conv1
3 bn_conv1
4 activation_1
5 maxpooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 merge_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 merge_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 merge_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 merge_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c


/opt/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from 0.41571 to 0.22631, saving model to ./checkpoints/checkpoint2/weights.000-0.2263.hdf5
15616/15583 [==============================] - 296s - loss: 0.2269 - acc: 0.9276 - val_loss: 0.2263 - val_acc: 0.9274
Epoch 2/300
15616/15583 [==============================] - 279s - loss: 0.1271 - acc: 0.9605 - val_loss: 0.1506 - val_acc: 0.9483
Epoch 3/300
15645/15583 [==============================] - 283s - loss: 0.0818 - acc: 0.9746 - val_loss: 0.1408 - val_acc: 0.9619
Epoch 4/300
15616/15583 [==============================] - 280s - loss: 0.0592 - acc: 0.9817 - val_loss: 0.0761 - val_acc: 0.9768
Epoch 5/300
15616/15583 [==============================] - 280s - loss: 0.0499 - acc: 0.9838 - val_loss: 0.0565 - val_acc: 0.9844
Epoch 6/300
15645/15583 [==============================] - 280s - loss: 0.0470 - acc: 0.9856 - val_loss: 0.0788 - val_acc: 0.9776
Epoch 7/300
15616/15583 [==============================] - 279s - loss: 0.0330 - acc: 0.9899 - val_loss: 0.061

KeyboardInterrupt: 

In [9]:
#resume training

files = glob.glob('./checkpoints/checkpoint2/*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
model = load_model(files[index])

model.fit_generator(train_generator, samples_per_epoch=len(X_train), nb_epoch=30, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=valid_generator, nb_val_samples=len(X_valid), nb_worker=3, pickle_safe=True)

Loading model from checkpoints file ./checkpoints/checkpoint2/weights.004-1.1775.hdf5
Epoch 1/30
15488/15583 [============================>.] - ETA: 1s - loss: 0.8447 - acc: 0.7795

/opt/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_loss improved from inf to 1.02681, saving model to ./checkpoints/checkpoint2/weights.000-1.0268.hdf5
15616/15583 [==============================] - 291s - loss: 0.8449 - acc: 0.7791 - val_loss: 1.0268 - val_acc: 0.7845
Epoch 2/30
15616/15583 [==============================] - 244s - loss: 0.7965 - acc: 0.7874 - val_loss: 0.9675 - val_acc: 0.7742
Epoch 3/30
15645/15583 [==============================] - 246s - loss: 0.7738 - acc: 0.7927 - val_loss: 0.8449 - val_acc: 0.7858
Epoch 4/30
15616/15583 [==============================] - 244s - loss: 0.7229 - acc: 0.8016 - val_loss: 0.7443 - val_acc: 0.7956
Epoch 5/30
15616/15583 [==============================] - 244s - loss: 0.6862 - acc: 0.8055 - val_loss: 0.6607 - val_acc: 0.8145
Epoch 6/30
15645/15583 [==============================] - 243s - loss: 0.6774 - acc: 0.8113 - val_loss: 0.6671 - val_acc: 0.8128
Epoch 7/30
15616/15583 [==============================] - 243s - loss: 0.6512 - acc: 0.8159 - val_loss: 0.6110 - val_ac

In [102]:
#get bbox from detections_full_AGNOSTICnms.pkl
RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'

import pickle 
with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
    detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    
outputs = []
count = np.zeros(len(detections_full_AGNOSTICnms))

for im in range(len(detections_full_AGNOSTICnms)):
    outputs_im = []
    detects_im = detections_full_AGNOSTICnms[im]
    for i in range(len(detects_im)):
        if np.max(detects_im[i,5:]) >= CONF_THRESH:
            outputs_im.append(detects_im[i,:]) 
    count[im] = len(outputs_im)
    if len(outputs_im) == 0:
        ind = np.argmax(np.max(detects_im[:,5:], axis=1))
        outputs_im.append(detects_im[ind,:])
    outputs_im = np.asarray(outputs_im)
    outputs.append(outputs_im)
    
#crop test images and cache to TEST_CROP_DIR

# if not os.path.exists(TEST_CROP_DIR):
#     os.mkdir(TEST_CROP_DIR)
# files = glob.glob(TEST_CROP_DIR+'*')
# for f in files:
#     os.remove(f)
    
# with open("../RFCN/ImageSets/Main/test.txt","r") as f:
#     ims = f.readlines()
# test_files = [im[:-1]+'.jpg' for im in ims]

# for i in range(len(outputs)):
#     if i%1000 == 0:
#         print(i)
#     filename = test_files[i]
#     bboxes = outputs[i]
#     basename, file_extension = os.path.splitext(filename) 
#     image = Image.open(TEST_DIR+filename)
#     for j in range(len(bboxes)):
#         bbox = bboxes[j]
#         xmin = bbox[0]
#         ymin = bbox[1]
#         xmax = bbox[2]
#         ymax = bbox[3]
#         file_crop = TEST_CROP_DIR+basename+'_{}'.format(j)+'.jpg'
#         cropped = image.crop((xmin, ymin, xmax, ymax))
#         width_cropped, height_cropped = cropped.size
#         if height_cropped > width_cropped: cropped = cropped.transpose(method=2)
#         cropped.save(file_crop)
        
print(sum([outputs[i].shape[0] for i in range(len(outputs))]))

6037


In [103]:
outputs[2]

array([[  6.04372192e+02,   9.93144073e+01,   9.23525879e+02,
          2.13722458e+02,   4.10640117e-04,   9.98496771e-01,
          9.39608843e-04,   7.38607469e-07,   6.02875275e-07,
          1.48173902e-04,   6.06100627e-07,   2.90451771e-06],
       [  3.12184143e+02,   1.27184914e+02,   6.84601257e+02,
          2.65319366e+02,   7.97489847e-05,   9.97261882e-01,
          2.54703895e-03,   2.05868233e-07,   4.01888570e-07,
          1.09348577e-04,   4.15622225e-07,   9.21043522e-07],
       [  9.27494385e+02,   1.43513062e+02,   1.19575684e+03,
          2.68516327e+02,   1.11477459e-02,   9.85716045e-01,
          4.04080289e-04,   3.50566006e-05,   7.49396277e-05,
          2.54306523e-03,   9.93645517e-06,   6.90411107e-05]], dtype=float32)

In [106]:
[outputs[i].shape[0] for i in range(3)]

[1, 1, 3]

In [63]:
collections.Counter(count)

Counter({0.0: 682,
         1.0: 3241,
         2.0: 617,
         3.0: 143,
         4.0: 50,
         5.0: 24,
         6.0: 14,
         7.0: 2,
         8.0: 3,
         9.0: 1})

In [67]:
sum(count)+682

6037.0

In [75]:
#Load test data

import datetime

def read_image(src):
    """Read and resize individual images"""
    im = Image.open(src)
    im = im.resize((COLS, ROWS), Image.BILINEAR)
    im = np.asarray(im)
    return im

if os.path.exists('../data/data_test_BBCrop_{}_{}.pickle'.format(ROWS, COLS)):
    print ('Exist data_test_BBCrop_{}_{}.pickle. Loading test data from file.'.format(ROWS, COLS))
    with open('../data/data_test_BBCrop_{}_{}.pickle'.format(ROWS, COLS), 'rb') as f:
        data_test = pickle.load(f)
    X_test_crop = data_test['X_test_crop']
    test_crop_files = data_test['test_crop_files']
else:
    print ('Loading test data from original images. Generating data_test_BBCrop_{}_{}.pickle.'.format(ROWS, COLS))

    test_crop_files = sorted([im for im in os.listdir(TEST_CROP_DIR)])
    X_test_crop = np.ndarray((len(test_crop_files), ROWS, COLS, 3), dtype=np.uint8)

    for i, im in enumerate(test_crop_files): 
        X_test_crop[i] = read_image(TEST_CROP_DIR+im)
        if i%1000 == 0: print('Processed {} of {}'.format(i, len(test_crop_files)))
            
    data_test = {'X_test_crop': X_test_crop,'test_crop_files': test_crop_files }
    
    with open('../data/data_test_BBCrop_{}_{}.pickle'.format(ROWS, COLS), 'wb') as f:
        pickle.dump(data_test, f, protocol=4)
        
X_test_crop = X_test_crop / 255.

Loading test data from original images. Generating data_test_BBCrop_224_224.pickle.
Processed 0 of 6037
Processed 1000 of 6037
Processed 2000 of 6037
Processed 3000 of 6037
Processed 4000 of 6037
Processed 5000 of 6037
Processed 6000 of 6037


In [76]:
test_crop_files

['img_00003_0_.jpg',
 'img_00003_1_.jpg',
 'img_00003_2_.jpg',
 'img_00003_3_.jpg',
 'img_00004_0_.jpg',
 'img_00005_0_.jpg',
 'img_00007_0_.jpg',
 'img_00008_0_.jpg',
 'img_00009_0_.jpg',
 'img_00009_1_.jpg',
 'img_00009_2_.jpg',
 'img_00010_0_.jpg',
 'img_00010_1_.jpg',
 'img_00011_0_.jpg',
 'img_00012_0_.jpg',
 'img_00014_0_.jpg',
 'img_00015_0_.jpg',
 'img_00018_0_.jpg',
 'img_00018_1_.jpg',
 'img_00019_0_.jpg',
 'img_00020_0_.jpg',
 'img_00020_1_.jpg',
 'img_00020_2_.jpg',
 'img_00020_3_.jpg',
 'img_00021_0_.jpg',
 'img_00022_0_.jpg',
 'img_00023_0_.jpg',
 'img_00027_0_.jpg',
 'img_00028_0_.jpg',
 'img_00029_0_.jpg',
 'img_00029_1_.jpg',
 'img_00030_0_.jpg',
 'img_00032_0_.jpg',
 'img_00033_0_.jpg',
 'img_00037_0_.jpg',
 'img_00038_0_.jpg',
 'img_00039_0_.jpg',
 'img_00040_0_.jpg',
 'img_00041_0_.jpg',
 'img_00041_1_.jpg',
 'img_00043_0_.jpg',
 'img_00045_0_.jpg',
 'img_00046_0_.jpg',
 'img_00047_0_.jpg',
 'img_00052_0_.jpg',
 'img_00053_0_.jpg',
 'img_00053_1_.jpg',
 'img_00055_0

In [111]:
#test preds clsMaxAve
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

# files = glob.glob('./checkpoints/checkpoint2/*')
# val_losses = [float(f.split('-')[-1][:-5]) for f in files]
# index = val_losses.index(min(val_losses))
# print('Loading model from', files[index])
# model = load_model(files[index])

#test_crop_preds = model.predict(X_test_crop, batch_size=BatchSize, verbose=1)
test_crop_preds = np.vstack(outputs)[:,4:]

with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims]

count = np.zeros(len(test_files))

test_preds = np.ndarray((len(test_files), test_crop_preds.shape[1]), dtype=np.float32)
#for j in range(len(test_files)):
for j in range(2,3):
    if j%1000 == 0:
        print(j)
    file = test_files[j]
    print(file)
    test_preds_im = []
    for i in range(len(test_crop_files)):
        if test_crop_files[i][:9] == file[:9]:
            print(i)
            test_preds_im.append(test_crop_preds[i])
    test_preds_im = np.asarray(test_preds_im)
    print(test_preds_im)
    score_max = np.max(test_preds_im, axis=1)
    print(score_max)
    inds = np.argmax(test_preds_im, axis=1)
    print(inds)
    labels = [FISH_CLASSES[ind] for ind in inds]
    columns = FISH_CLASSES[:]
    test_preds_im_df = pd.DataFrame(test_preds_im, columns=columns)
    test_preds_im_df['max_cls'] = labels
    test_preds_im_df['max_score'] = score_max 
    test_preds_im_df['Counts'] = test_preds_im_df.groupby(['max_cls'])['max_cls'].transform('count')
    idx = test_preds_im_df.groupby(['max_cls'])['max_score'].transform(max) == test_preds_im_df['max_score']
    test_preds_im_df = test_preds_im_df[idx]
    count[j] = test_preds_im_df.shape[0]
    l = FISH_CLASSES.copy()
    l.append('Counts')
    test_preds_im_array = test_preds_im_df[l].as_matrix() 
    print(test_preds_im_array)
    test_preds[j] = np.average(test_preds_im_array[:,:-1], axis=0, weights=test_preds_im_array[:,-1], returned=False)
    print(test_preds[j])

img_00009.jpg
8
9
10
[[  9.99980569e-01   8.15407475e-06   3.12006108e-07   4.17473274e-07
    5.70212205e-06   3.34142851e-06   7.31538478e-07   7.74503690e-07]
 [  1.43180434e-02   1.84875142e-04   8.17218643e-06   4.66901227e-04
    2.37479799e-05   3.30876745e-02   9.51784730e-01   1.25874343e-04]
 [  9.99998331e-01   1.45928971e-06   1.26569386e-08   3.44543132e-08
    1.20649318e-07   1.12331051e-07   4.56219169e-08   4.73076369e-08]]
[ 0.99998057  0.95178473  0.99999833]
[0 6 0]
[[  1.43180434e-02   1.84875142e-04   8.17218643e-06   4.66901227e-04
    2.37479799e-05   3.30876745e-02   9.51784730e-01   1.25874343e-04
    1.00000000e+00]
 [  9.99998331e-01   1.45928971e-06   1.26569386e-08   3.44543132e-08
    1.20649318e-07   1.12331051e-07   4.56219169e-08   4.73076369e-08
    2.00000000e+00]]
[  6.71438217e-01   6.25979083e-05   2.73250021e-06   1.55656715e-04
   7.99642567e-06   1.10292993e-02   3.17261606e-01   4.19896533e-05]


In [87]:
collections.Counter(count)
#Counter({0.0: 636, 1.0: 4079, 2.0: 62})

Counter({1.0: 4248, 2.0: 444, 3.0: 67, 4.0: 18})

In [108]:
test_crop_preds[:5,:]

array([[  9.99996901e-01,   1.53745088e-06,   7.76591378e-08,
          2.14523766e-07,   5.99420616e-07,   2.10384329e-07,
          1.28219966e-07,   2.08938104e-07],
       [  1.26663246e-04,   8.30609642e-05,   6.00004423e-05,
          3.26823488e-06,   8.62986838e-07,   9.32441822e-08,
          3.09665552e-06,   9.99722898e-01],
       [  4.10640117e-04,   9.98496771e-01,   9.39608843e-04,
          7.38607469e-07,   6.02875275e-07,   1.48173902e-04,
          6.06100627e-07,   2.90451771e-06],
       [  7.97489847e-05,   9.97261882e-01,   2.54703895e-03,
          2.05868233e-07,   4.01888570e-07,   1.09348577e-04,
          4.15622225e-07,   9.21043522e-07],
       [  1.11477459e-02,   9.85716045e-01,   4.04080289e-04,
          3.50566006e-05,   7.49396277e-05,   2.54306523e-03,
          9.93645517e-06,   6.90411107e-05]], dtype=float32)

In [116]:
test_files[:5]

['img_00005.jpg',
 'img_00007.jpg',
 'img_00009.jpg',
 'img_00018.jpg',
 'img_00027.jpg']

In [78]:
test_preds

array([[  2.75402388e-04,   9.99508739e-01,   1.94741078e-04, ...,
          1.92895932e-05,   1.39052901e-07,   1.36287781e-06],
       [  1.38710209e-04,   9.99382854e-01,   1.62361772e-04, ...,
          3.00128188e-04,   1.35484186e-06,   9.69768553e-06],
       [  6.71438217e-01,   6.25979083e-05,   2.73250021e-06, ...,
          1.10292993e-02,   3.17261606e-01,   4.19896533e-05],
       ..., 
       [  4.99234915e-01,   6.34203083e-04,   8.27453114e-05, ...,
          5.71008386e-05,   5.37206279e-06,   1.73051958e-05],
       [  6.59573881e-04,   1.74273780e-04,   6.02989021e-05, ...,
          9.98909950e-01,   4.62454700e-05,   3.06304145e-07],
       [  1.19337077e-04,   2.77301551e-05,   2.62545818e-05, ...,
          9.99791563e-01,   4.65846051e-06,   1.71323211e-06]], dtype=float32)

In [79]:
#temperature
T = 2.5
test_preds_T = np.exp(np.log(test_preds)/T)
test_preds_T = test_preds_T/np.sum(test_preds_T, axis=1, keepdims=True)

In [80]:
#calculate train logloss
#FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

train_files = test_files[1000:]
train_preds = test_preds_T[1000:,:]
with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
    train_file_labels = f.readlines()

log_losses = []
for i in range(len(train_preds)):
    im = train_files[i][:-4]
    for im_label in train_file_labels:
        if im_label[:9] == im:
            label = im_label[10:-1]
            index = FISH_CLASSES.index(label)
            log_losses.append(-math.log(train_preds[i,index]))
log_loss = sum(log_losses) / float(len(log_losses))
print('logloss of train is', log_loss )

logloss of train is 3.094302323525866


In [53]:
#test submission
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(test_preds_T[:1000,:], columns=FISH_CLASSES)
submission.insert(0, 'image', test_files[:1000])

info = 'RFCN_AGONOSTICnms_'+RFCN_MODEL+'_BBCROP_resnet50_clsMaxAve_conf{:.2f}_T{}_'.format(CONF_THRESH, T) + '{:.4f}'.format(log_loss)
sub_file = 'submission_' + info + '.csv'
submission.to_csv(sub_file, index=False)

In [ ]:
###clear checkpoints folder

if not os.path.exists('./checkpoints'):
    os.mkdir('./checkpoints')
files = glob.glob('./checkpoints/*')
for f in files:
    os.remove(f)

In [ ]:
###clear logs folder

if not os.path.exists('./logs'):
    os.mkdir('./logs')
files = glob.glob('./logs/*')
for f in files:
    os.remove(f)